In [23]:
import keras 
import numpy as np
import tensorflow as tf

import pandas as pd
import statsmodels.api

from sklearn.metrics import r2_score, mean_squared_error

In [24]:
data = statsmodels.api.datasets.get_rdataset('mtcars').data

In [25]:
X = data.drop(columns=['mpg'])
y = data['mpg'].values.reshape(-1, 1)

In [26]:
X.shape

(32, 10)

### Seleçao de modelos

In [27]:
def modelo(x, w_):
    return x @ w_

def weights_bias(X:np.ndarray, y:np.ndarray):
    return np.linalg.inv( np.dot(X.T, X) ) @ X.T @ y

In [28]:
pesos = weights_bias(X.values, y)
pesos.T

array([[ 0.35082641,  0.01354278, -0.02054767,  1.24158213, -3.8261315 ,
         1.19139689,  0.18972068,  2.8322223 ,  1.05426253, -0.26321386]])

In [29]:
modelo(X.values, pesos)[0:2, :],y[0:2,:]

(array([[22.43607595],
        [22.12759467]]),
 array([[21.],
        [21.]]))

#### Backward Elimination

In [30]:
def backwardEliminação(X, y, data, target_name='mpg', significancia=0.05):
    AIC = np.inf
    columnsList = data.drop(columns=[target_name]).columns.tolist()
    ant_ = len(columnsList)

    for col in data.columns:
        model = statsmodels.api.OLS(y, X[columnsList].values).fit()
        
        strCol = ' '.join(columnsList) + " | AIC="
        if len(columnsList) == ant_:
            print(f'{strCol}{round(model.aic, 2)}')

        index = pd.Series(model.pvalues).idxmax()
        
        if model.pvalues[index] > significancia:
            print(f'Remove {columnsList[index]}')
            del columnsList[index]
        
    return model

In [31]:
model = backwardEliminação(X, y, data, significancia=0.1)

cyl disp hp drat wt qsec vs am gear carb | AIC=160.36
Remove vs
Remove carb
Remove cyl
Remove gear
Remove hp
Remove drat
Remove disp


In [32]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.987
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                              741.0
Date:                Sun, 10 Nov 2024   Prob (F-statistic):                    1.71e-27
Time:                        10:40:44   Log-Likelihood:                         -73.115
No. Observations:                  32   AIC:                                      152.2
Df Residuals:                      29   BIC:                                      156.6
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -3.1855      0.483     -6.598      0.000      -4.173      -2.198
x2             1.5998      0.102     15.665      0.000       1.391       1.809
x3             4.2995      1.024      4.198      0.000       2.205       6.394
==============================================================================
Omnibus:                        2.348   Durbin-Watson:                   1.861
Prob(Omnibus):                  0.309   Jarque-Bera (JB):                1.351
Skew:                           0.177   Prob(JB):                        0.509
Kurtosis:                       2.057   Cond. No.                         43.8
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Regularização L1 Lasso

In [33]:
def modelo(x, weights, bias):
    return tf.matmul(x, weights) + bias

In [34]:
def MSE_loss(ytrue:tf.Tensor, ypred:tf.Tensor):
    return tf.reduce_mean(
        tf.pow((ytrue - ypred), 2)
    )

In [35]:
X_ = X.values.astype(np.float32)
y_ = y.astype(np.float32)

In [36]:
X_ = (X_ - X_.mean(axis=0))/X_.std(axis=0)

In [37]:
dataset = tf.data.Dataset.from_tensor_slices((X_, y_))

In [38]:
train_ = dataset\
        .batch(batch_size=32)\
        .shuffle(buffer_size=10, seed=10)

In [45]:
tf.random.set_seed( seed = 1 )
# pesos
weights = tf.Variable(
    tf.zeros((10, 1)), dtype=tf.float32
    )
# bias
bias = tf.Variable(
    [[0]], dtype=tf.float32
    )

optim = keras.optimizers.SGD(learning_rate=0.01, momentum=0.99, nesterov=True)
lambda_ = 0.1

for epoch in range(10):
    for xbatch, ybatch in train_:
        with tf.GradientTape() as tape:
            pred = modelo(xbatch, weights, bias)

            # Expandindo  
            tensor_ = tf.tile(bias, [10,1])
            weights_i = tf.concat([weights, tensor_], axis=0)[:11, :]

            # Penalidade lambda * SUM |Wi|
            L1 = lambda_ * tf.reduce_sum(tf.abs(weights))
            
            # calculando a perda
            loss = MSE_loss(ybatch, pred) + L1
        
        # adicioando o gradient
        gradient = tape.gradient(loss, [weights, bias])
        optim.apply_gradients(
            zip(gradient, [weights, bias])
        )
    print(f'Epoch : {epoch}\n =>loss : {loss:.2f}| l1: {L1:.2f}')
    

Epoch : 0
 =>loss : 438.82| l1: 0.00
Epoch : 1
 =>loss : 395.65| l1: 0.16
Epoch : 2
 =>loss : 341.85| l1: 0.36
Epoch : 3
 =>loss : 284.30| l1: 0.57
Epoch : 4
 =>loss : 227.80| l1: 0.77
Epoch : 5
 =>loss : 174.97| l1: 0.93
Epoch : 6
 =>loss : 127.08| l1: 1.05
Epoch : 7
 =>loss : 85.15| l1: 1.11
Epoch : 8
 =>loss : 50.65| l1: 1.12
Epoch : 9
 =>loss : 25.56| l1: 1.07


In [46]:
weights, bias

(<tf.Variable 'Variable:0' shape=(10, 1) dtype=float32, numpy=
 array([[-1.1231992 ],
        [-1.1466397 ],
        [-1.182853  ],
        [ 0.9152894 ],
        [-1.5939044 ],
        [ 0.37498504],
        [ 0.69366896],
        [ 1.1014943 ],
        [ 0.6130859 ],
        [-1.125236  ]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(1, 1) dtype=float32, numpy=array([[19.290466]], dtype=float32)>)

In [48]:
pred = modelo(X_, weights, bias)

In [49]:
mean_squared_error(y_, pred)

np.float32(10.844315)

In [50]:
r2_score(y_, pred.numpy())

0.691826343536377

#### L2 Ridge

In [66]:
tf.random.set_seed( seed = 1 )
# pesos
weights = tf.Variable(
    tf.zeros((10, 1)), dtype=tf.float32
    )
# bias
bias = tf.Variable(
    [[0]], dtype=tf.float32
    )

optim = keras.optimizers.SGD(learning_rate=0.01, momentum=0.99, nesterov=True)
lambda_ = 0.20

for epoch in range(10):
    for xbatch, ybatch in train_:
        with tf.GradientTape() as tape:
            pred = modelo(xbatch, weights, bias)

            # Expandindo  
            tensor_ = tf.tile(bias, [10,1])
            weights_i = tf.concat([weights, tensor_], axis=0)[:11, :]

            # Penalidade lambda * SUM |Wi|
            L1 = lambda_ * tf.reduce_sum(tf.pow(weights, 2))
            
            # calculando a perda
            loss = MSE_loss(ybatch, pred) + L1
        
        # adicioando o gradient
        gradient = tape.gradient(loss, [weights, bias])
        optim.apply_gradients(
            zip(gradient, [weights, bias])
        )
    print(f'Epoch : {epoch}\n\t=>loss : {loss:.2f} & l1: {L1:.2f}')
    

Epoch : 0
	=>loss : 438.82 & l1: 0.00
Epoch : 1
	=>loss : 395.54 & l1: 0.05
Epoch : 2
	=>loss : 341.74 & l1: 0.27
Epoch : 3
	=>loss : 284.42 & l1: 0.68
Epoch : 4
	=>loss : 228.25 & l1: 1.24
Epoch : 5
	=>loss : 175.63 & l1: 1.81
Epoch : 6
	=>loss : 127.69 & l1: 2.27
Epoch : 7
	=>loss : 85.46 & l1: 2.52
Epoch : 8
	=>loss : 50.58 & l1: 2.52
Epoch : 9
	=>loss : 25.23 & l1: 2.32


In [59]:
weights

<tf.Variable 'Variable:0' shape=(10, 1) dtype=float32, numpy=
array([[-1.0646281 ],
       [-1.0882386 ],
       [-1.1273296 ],
       [ 0.87819165],
       [-1.5153953 ],
       [ 0.37788078],
       [ 0.6662864 ],
       [ 1.0623733 ],
       [ 0.6029124 ],
       [-1.0876013 ]], dtype=float32)>

In [60]:
pred = modelo(X_, weights, bias)

In [61]:
mean_squared_error(y_, pred)

np.float32(9.510708)

In [62]:
r2_score(y_, pred.numpy())

0.7297247648239136